In [1]:
import os

import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupKFold

from tqdm import tqdm
from random import choices
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

2023-06-03 21:51:41.496902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-03 21:51:42.575716: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-03 21:51:42.575792: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-06-03 21:51:42.575802: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [3]:
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import StandardScaler
all_data = pd.read_csv('../data/forauto.csv')
useless_cols = ['year_weekday', 'Longitude','Uid_count','img_length','img_width', 'pixel','Category.1', 
                'Concept.1', 'Subcategory.1', 'Alltags','Title','img_file','firstdate','firstweek', 'firstmonth', 
                'firstdatetaken', 'firstdatetakenweek','firstdatetakenmonth','totalViews']
cate_cols = ['Uid', 'Category', 'Latitude', 'Geoaccuracy', 'Subcategory', 'Concept', 'Mediatype', 
             'hour', 'day', 'weekday', 'week_hour','Geoaccuracy', 'ispro' , 'Ispublic', 'img_model']
useless_cols += cate_cols
all_data = all_data.drop(useless_cols,axis=1)

rob = pd.read_csv('./preRoberta_fea.csv')
all_data = pd.concat([all_data,rob],axis=1)
columns = ['Title_len', 'Title_number', 'Alltags_len', 'Alltags_number', 
           'photo_count', 'totalTags', 'totalGeotagged', 'totalFaves',
          'totalInGroup','photoCount','meanView', 'meanTags', 'meanFaves', 'followerCount','followingCount']


for i in columns:
    all_data[i] = np.log1p(all_data[i])

for c in columns:
    prep = StandardScaler()
    all_data[c] = prep.fit_transform(all_data[[c]])

train = all_data[:-180581]
test = all_data[-180581:]

test = test.reset_index(drop=True)

y = train[['label']].values
X = train.drop(['label'],axis=1).values
test_df = test.drop(['label'],axis=1).values

In [4]:
def create_model():
    model = keras.Sequential([
        layers.Dense(4096, activation='relu', input_shape=(823,)),
        layers.Reshape((256, 16)),
        layers.Conv1D(filters=16, kernel_size=5, strides=1, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='linear'),
    ])
    
    model.compile(
        optimizer=tf.optimizers.Adam(1e-4),
        loss='mse',
        metrics=[keras.metrics.RootMeanSquaredError()]
    )
    
    return model

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K
class GCF:

    N_TRAIN = 1_500_000
    N_FOLDS = 5
    
    SEED = 0
    
    N_EPOCHS = 1000
    BATCH_SIZE = 4096
    EARLY_STOPPING_PATIENCE = 10
    EARLY_STOPPING_MIN_DELTA = 1e-3

In [8]:
from sklearn.model_selection import StratifiedKFold, KFold
from scipy import stats
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
from scipy import stats
rmse_lst, score_lst = [], []
y_preds = []
# models = []

kfold = KFold(n_splits=5, shuffle=True, random_state=2020)
fold = 0

for train_idx, valid_idx in kfold.split(X, y):
    model = create_model()
    
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='root_mean_squared_error',
        patience=GCF.EARLY_STOPPING_PATIENCE,
        min_delta=GCF.EARLY_STOPPING_MIN_DELTA,
        restore_best_weights=True,
    )
    reduce_lr = ReduceLROnPlateau(
                        monitor='root_mean_squared_error',
                        factor=0.5,
                        patience=3,
                        min_lr=1e-5,
                        verbose=1
    )

    history = model.fit(
        X[train_idx, :], y[train_idx],
        validation_data=(X[valid_idx, :], y[valid_idx]),
        batch_size=GCF.BATCH_SIZE,
        epochs=GCF.N_EPOCHS,
        callbacks=[early_stopping, reduce_lr],
    )
    

    valid_pred = model.predict(X[valid_idx, :]).reshape(1, -1)[0]
    valid_mse = mean_squared_error(y[valid_idx], valid_pred)
    valid_mae = mean_absolute_error(y[valid_idx], valid_pred)
    valid_src = stats.spearmanr(y[valid_idx], valid_pred)[0]

    print(f'Fold-{fold}: RMSE={valid_mse}, MAE={valid_mae},SRC={valid_src} ')
    
    y_pred = model.predict(test_df).reshape(1, -1)[0]
    y_preds.append(y_pred)
    # models.append(model)
    model.save(f"./ump_1dcnn_f{fold}.h5")
    rmse_lst.append(valid_mse)
    fold += 1

Epoch 1/1000
60/60 [==============================] - 4s 30ms/step - loss: 14.2584 - root_mean_squared_error: 3.7760 - val_loss: 5.3945 - val_root_mean_squared_error: 2.3226 - lr: 1.0000e-04
Epoch 2/1000
60/60 [==============================] - 1s 20ms/step - loss: 5.0318 - root_mean_squared_error: 2.2432 - val_loss: 4.7203 - val_root_mean_squared_error: 2.1726 - lr: 1.0000e-04
Epoch 3/1000
60/60 [==============================] - 1s 20ms/step - loss: 4.5355 - root_mean_squared_error: 2.1297 - val_loss: 4.3336 - val_root_mean_squared_error: 2.0817 - lr: 1.0000e-04
Epoch 4/1000
60/60 [==============================] - 1s 20ms/step - loss: 4.1979 - root_mean_squared_error: 2.0489 - val_loss: 4.0418 - val_root_mean_squared_error: 2.0104 - lr: 1.0000e-04
Epoch 5/1000
60/60 [==============================] - 1s 20ms/step - loss: 3.9219 - root_mean_squared_error: 1.9804 - val_loss: 3.7901 - val_root_mean_squared_error: 1.9468 - lr: 1.0000e-04
Epoch 6/1000
60/60 [=============================

In [9]:
y_preds

[array([ 8.204643 , 11.39289  , 10.227794 , ...,  8.0059805,  9.736052 ,
         7.770362 ], dtype=float32),
 array([ 8.4395   , 12.257613 ,  9.84161  , ...,  8.454668 ,  9.591785 ,
         7.9210997], dtype=float32),
 array([ 9.579706 , 10.611546 ,  9.4129715, ...,  9.457863 ,  9.029177 ,
         8.335367 ], dtype=float32),
 array([ 9.401019, 11.260335,  9.364364, ...,  8.36637 ,  9.822805,
         7.443412], dtype=float32),
 array([ 8.927393 , 11.636412 , 11.151912 , ...,  8.979655 ,  9.157673 ,
         7.7556987], dtype=float32)]

In [19]:
pd.DataFrame(y_preds[4],columns=['1dcnn_4']).to_csv('./1dcnn_4.csv',header=True,index=None)

In [20]:
valid_ans = np.mean(y_preds, axis=0)
valid_ans

array([ 8.910452, 11.431758,  9.999731, ...,  8.652907,  9.467498,
        7.845188], dtype=float32)

In [21]:
pd.DataFrame(valid_ans,columns=['1dcnn_mean']).to_csv('./1dcnn_mean.csv',header=True,index=None)

In [10]:
df_preds = pd.DataFrame(y_preds)
df_preds

,0,1,2,3,4,5,6,7,8,9,...,180571,180572,180573,180574,180575,180576,180577,180578,180579,180580
0,8.204643,11.392890,10.227794,6.983021,6.983021,6.983021,6.722751,6.725307,6.758959,6.733967,...,6.250739,8.843910,6.120072,5.993510,6.665653,7.995525,6.163742,8.005980,9.736052,7.770362
1,8.439500,12.257613,9.841610,6.797660,6.797660,6.797660,6.659466,6.644850,6.567548,6.659066,...,6.380440,7.826139,6.220476,6.695003,6.699414,7.321749,7.279061,8.454668,9.591785,7.921100
2,9.579706,10.611546,9.412971,6.486055,6.486055,6.486055,6.603312,6.606183,6.590973,6.586757,...,7.088863,7.677083,4.512009,4.750963,5.548397,7.842870,5.212536,9.457863,9.029177,8.335367
3,9.401019,11.260335,9.364364,6.888610,6.888610,6.888610,6.656661,6.631947,6.529427,6.644340,...,6.425628,8.373248,5.207808,6.155601,6.639772,8.821942,5.270638,8.366370,9.822805,7.443412
4,8.927393,11.636412,11.151912,6.707948,6.707948,6.707948,6.645917,6.683570,6.746226,6.649634,...,4.633233,8.554185,6.415594,6.233513,6.579780,7.696683,7.159212,8.979655,9.157673,7.755699
